In [2]:
import pandas as pd
import numpy as np

In [3]:
class Display(object):
    """
        Displaying HTML represantation of multiple object
    """
    template = """
        <div style = "float: left; padding: 10px;">
        <p style = "font-family: 'Courier New', Courier, monospace">{0}{1}
    """
    
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)
    
    def __repr__(self) -> str:
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)
    
    

In [4]:
df1 = pd.DataFrame(
    {
        'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
        'group': ['Accounting', 'Engineering', 'Engineering', 'HR']
    }
)
df2 = pd.DataFrame(
    {'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
     'hire_date': [2004, 2008, 2012, 2014]
    }
)

Display('df1', 'df2')
# print(df1)
# print(df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


# One-to-One Joins

To combine this information into a single DataFrame, we can use the pd.merge
function:

In [5]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


# Many-to-One Joins

In [6]:
df4 = pd.DataFrame(
    {
        'group': ['Accounting', 'Engineering', 'HR'],
        'supervisor': ['Carly', 'Guido', 'Steve']
    }
)

Display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

# Many-to-Many Joins


In [7]:
df5 = pd.DataFrame(
    {
        'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
        'skills': ['math', 'spreadsheets', 'software', 'math', 'spreadsheets', 'organization']
    }
)

Display('df1', 'df5', 'pd.merge(df1, df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,software
3,Engineering,math
4,HR,spreadsheets


# The on Keyword

Most simply, you can explicitly specify the name of the key column using the on keyword,
which takes a column name or a list of column names:

In [8]:
Display('df1', 'df2', "pd.merge(df1, df2, on = 'employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on = 'employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

# The left_on and right_on Keywords
At times you may wish to merge two datasets with different column names; for example,
we may have a dataset in which the employee name is labeled as “name” rather
than “employee”. In this case, we can use the left_on and right_on keywords to
specify the two column names:

In [25]:
df6 = pd.DataFrame({
    'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'salary': [70000, 80000, 120000, 90000]
})

pd.merge(df1, df6, left_on = 'employee', right_on = 'name')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


The result has a redundant column that we can drop if desired—for example, by
using the DataFrame.drop() method:

In [30]:
pd.merge(df1, df6, left_on = 'employee', right_on = 'name').drop('name', axis = 1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


If you’d like to mix indices and columns, you can combine left_index with right_on
or left_on with right_index to get the desired behavior:

In [31]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
Display('df1a', 'df6', "pd.merge(df1a, df3, left_index=True, right_on='name')")

KeyError: 'name'

KeyError: 'name'